### Regularized Learning Objective    
&emsp;&emsp;For a given data set with n examples and m features $ D=\left\{\left(\mathbf{x}_{i}, y_{i}\right)\right\}\left(|D|=n, \mathbf{x}_{i} \in \mathbb{R}^{m}, y_{i} \in \mathbb{R}\right)$,
a tree ensemble model (shown in Fig 1) uses K additive functions to predict the output      
$$ \hat{y}_{i}=\phi\left(\mathbf{x}_{i}\right)=\sum_{k=1}^{K} f_{k}\left(\mathbf{x}_{i}\right), \quad f_{k} \in \mathcal{F} \tag{1} $$        

<img src='../../Other/img/xgboost0.png' style="width:650px;height:400px">

where $ \mathcal{F}=\left\{f(\mathbf{x})=w_{q(\mathbf{x})}\right\}\left(q: \mathbb{R}^{m} \rightarrow T, w \in \mathbb{R}^{T}\right) $ is the space of regression trees (also known as CART).
Here q represents the structure of each tree that maps an example to the corresponding leaf index.
$T$ is the number of leaves in the tree. Each $ f_k$ corresponds to an independent tree structure $q$ and leaf weights $w$.
Unlike decision trees, each regression tree contains a continuous score on each of the leaf,we use $w_i$ to represent score on $ i$-th leaf.
For a given example, we will use the decision rules in the trees (given by $q$) to classify it into the leaves and calculate the
final prediction by summing up the score in the corresponding leaves (given by $w$).To learn the set of functions used in the model, we minimize the following regularized objective.

\begin{array}{l}
\mathcal{L}(\phi)=\sum_{i} l\left(\hat{y}_{i}, y_{i}\right)+\sum_{k} \Omega\left(f_{k}\right) \\ \tag{2}
\text { where } \Omega(f)=\gamma T+\frac{1}{2} \lambda\|w\|^{2}
\end{array}        

Here $l$  is a differentiable convex loss function that measures the difference between the prediction $\hat{y}_i$ and the target $ y_i$.The second term $\Omega $ penalizes the complexity of the model (i.e., the regression tree functions).The additional regularization term helps to smooth the final learnt weights to avoid over-


### Gradient Tree Boosting
&emsp;&emsp;The tree ensemble model in Eq (2) includes functions as parameters and cannot be optimized using traditional optimization methods in Euclidean space.
Instead, the model is trained in an additive manner. Formally, let $ \hat{y}_i $ be the prediction of the $i$-th instance at the $t$-th iteration,
we will need to add $f_t$ to minimize the following objective.            
$$\mathcal{L}^{(t)}=\sum_{i=1}^{n} l\left(y_{i}, | \hat{y}_{i}^{(t-1)}+f_{t}\left(\mathbf{x}_{i}\right)\right)+\Omega\left(f_{t}\right)$$              
This means we greedily add the $f_t$ that most improves our model according to Eq (2). To quickly optimize the objective in the general setting,
we approximate it using the second order Taylor expansion.         
$$ \mathcal{L}^{(t)} \simeq \sum_{i=1}^{n}\left[l\left(y_{i}, \hat{y}^{(t-1)}\right)+g_{i} f_{t}\left(\mathbf{x}_{i}\right)+\frac{1}{2} h_{i} f_{t}^{2}\left(\mathbf{x}_{i}\right)\right] + \Omega\left(f_{t}\right) $$                
where$g_{i}= \frac{\partial l\left(y_{i}, \hat{y}^{(t-1)}\right)}{\partial \hat{y}^{(t-1)}} $ and $h_{i}= \frac{\partial^2 l\left(y_{i}, \hat{y}^{(t-1)}\right)}{\partial \hat{y}^{(t-1)} \partial \hat{y}^{(t-1)}}$ are first and second order gradient statistics on the loss function.
We can remove the constant terms to obtain the following simplified objective at step $t$.            
$$ \tilde{\mathcal{L}}^{(t)}=\sum_{i=1}^{n}\left[g_{i} f_{t}\left(\mathbf{x}_{i}\right)+\frac{1}{2} h_{i} f_{t}^{2}\left(\mathbf{x}_{i}\right)\right]+\Omega\left(f_{t}\right) \tag{3} $$                
&emsp;&emsp;Define $I_{j}=\left\{i | q\left(\mathbf{x}_{i}\right)=j\right\}$ as the instance set of leaf $j$. We can rewrite Eq (3) by expanding the regularization term $\Omega$ as follows       

\begin{align}
\tilde{\mathcal{L}}^{(t)} &=\sum_{i=1}^{n}\left[g_{i} f_{t}\left(\mathbf{x}_{i}\right)+\frac{1}{2} h_{i} f_{t}^{2}\left(\mathbf{x}_{i}\right)\right]+\gamma T+\frac{1}{2} \lambda \sum_{j=1}^{T} w_{j}^{2} \\
&=\sum_{i=1}^{n}\left[g_{i} w_{q}\left(\mathbf{x}_{i}\right)+\frac{1}{2} h_{i} w_{q}^{2}\left(\mathbf{x}_{i}\right)\right]+\gamma T+\frac{1}{2} \lambda \sum_{j=1}^{T} w_{j}^{2} \\ 
&=\sum_{j=1}^{T}\left[\left(\sum_{i \in I_{j}} g_{i}\right) w_{j}+\frac{1}{2}\left(\sum_{i \in I_{j}} h_{i}+\lambda\right) w_{j}^{2}\right]+\gamma T 
\tag{4}
\end{align}                

&emsp;&emsp;For a fixed structure $q(x)$, we can compute the optimal weight $w_j^* $ of leaf $j$ by                
$$ w_{j}^{*}=-\frac{\sum_{i \in I_{j}} g_{i}}{\sum_{i \in I_{j}} h_{i}+\lambda} \tag{5} $$         
and calculate the corresponding optimal objective function value by          
$$\tilde{\mathcal{L}}^{(t)}(q)=-\frac{1}{2} \sum_{j=1}^{T} \frac{\left(\sum_{i \in I_{j}} g_{i}\right)^{2}}{\sum_{i \in I_{j}} h_{i}+\lambda}+\gamma T \tag{6} $$              
&emsp;&emsp;Eq (6) can be used as a scoring function to measure the quality of a tree structure $q$. This score is like the impurity score for evaluating decision trees,
except that it is derived for a wider range of objective functions.Fig 2 illustrates how this score can be calculated.            

<img src='../../Other/img/xgboost1.png' style="width:650px;height:400px">

&emsp;&emsp;Normally it is impossible to enumerate all the possible tree structures $q$. 
A greedy algorithm that starts from a single leaf and iteratively adds branches to the tree is used instead. 
Assume that $I_L$ and $I_R$ are the instance sets of left and right nodes after the split. 
Letting $ I = I_L \cup I_R$, then the loss reduction after the split is given by           
$$\mathcal{L}_{s p l i t}=\frac{1}{2}\left[\frac{\left(\sum_{i \in I_{L}} g_{i}\right)^{2}}{\sum_{i \in I_{L} } h_{i}+\lambda}+\frac{\left(\sum_{i \in I_{R}} g_{i}\right)^{2}}{\sum_{i \in I_{R}} h_{i}+\lambda}-\frac{\left(\sum_{i \in I} g_{i}\right)^{2}}{\sum_{i \in I} h_{i}+\lambda}\right]-\gamma \tag{7}  $$           
Let us define $ G_{j}=\sum_{i \in I_{j}} g_{i} \quad H_{j}=\sum_{i \in I_{j}} h_{i}$                
$$\mathcal{L}_{s p l i t}=G a i n=\frac{1}{2}\left[\frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{\left(G_{L}+G_{R}\right)^{2}}{H_{L}+H_{R}+\lambda}\right] - \gamma $$           
$ \frac{G_{L}^{2}}{H_{L}+\lambda}  $:the score of left child;$ \frac{G_{R}^{2}}{H_{R}+\lambda}  $:the score of right child;$\frac{\left(G_{L}+G_{R}\right)^{2}}{H_{L}+H_{R}+\lambda}$:the score of if we do not split;
$ \gamma $:The complexity cost by introducing additional leaf.               
This formula is usually used in practice for evaluating the split candidates.      


### Shrinkage and Column Subsampling

&emsp;&emsp;Besides the regularized objective mentioned, 
two additional techniques are used to further prevent over-fitting. The first technique is shrinkage introduced by Friedman(This is learning_rate parameters in the GBDT).
Shrinkage scales newly added weights by a factor $  \eta$ after each step of tree boosting. 
Similar to a learning rate in stochastic optimization, shrinkage reduces the influence of each individual tree and leaves space for future trees to improve the model.
The second technique is column (feature) subsampling. This technique is commonly used in RandomForest. 
According to user feedback, using column sub-sampling prevents over-fitting even more so than the traditional row sub-sampling(This is subsample parameters in the GBDT)(which is also supported).
The usage of column sub-samples also speeds up computations of the parallel algorithm. 


### Basic Exact Greedy Algorithm
&emsp;&emsp;One of the key problems in tree learning is to find the best split as indicated by Eq (7). 
In order to do so, a split finding algorithm enumerates over all the possible splits on all the features.
We call this the exact greedy algorithm. Most existing single machine tree boosting implementations.
The exact greedy algorithm is shown in Alg 1. 
It is computationally demanding to enumerate all the possible splits for continuous features. 
In order to do so efficiently, the algorithm must first sort the data according to feature values and visit the data in sorted order to accumulate the gradient statistics for the structure score in Eq (7).    

***

**Algorithm 1**: Exact Greedy Algorithm for Split Finding

**Input**:$ I$, instance set of current node
**Input**: $d$, feature dimension

gain $\leftarrow 0 $

$G \leftarrow \sum_{i \in I} g_{i}, H \leftarrow \sum_{i \in I} h_{i}$

**for** $k = 1$ **to** $m$ **do**

&emsp;&emsp;$G_{L} \leftarrow 0, H_{L} \leftarrow 0$

&emsp;&emsp;**for** $j$ $in$ $sorted(I, \text{by} \, \mathbf{x}_{jk})$ **do**

&emsp;&emsp;&emsp;&emsp;$G_{L} \leftarrow G_{L}+g_{j}, \quad H_{L} \leftarrow H_{L}+h_{j}$

&emsp;&emsp;&emsp;&emsp;$G_{R} \leftarrow G-G_{L}, H_{R} \leftarrow H-H_{L}$

&emsp;&emsp;&emsp;&emsp;$\text {score} \leftarrow \max \left(\text {score}, \frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{G^{2}}{H+\lambda}\right)$

&emsp;&emsp;&emsp;&emsp;**end**

&emsp;&emsp;**end**

**Output**: Split with max score

***


### Approximate Algorithm
&emsp;&emsp;The exact greedy algorithm is very powerful since it enumerates over all possible splitting points greedily. 
However, it is impossible to efficiently do so when the data does not fit entirely into memory. 
Same problem also arises in the distributed setting. in these two settings, an approximate algorithm needs to be used. 
An approximate framework for tree boosting is given in Alg 2. To summarize, the algorithm first proposes candidate splitting points according to percentiles of feature distribution. 
The algorithm then maps the continuous features into buckets split by these candidate points, 
aggregates the statistics and finds the best solution among proposals based on the aggregated statistics.           
&emsp;&emsp;There are two variants of the algorithm, depending on when the proposal is given. 
The global variant proposes all the candidate splits during the initial phase of tree construction and uses the same proposals for split finding at all levels. 
The local variant re-proposes after each split. The global method requires fewer proposal steps than the local method. 
However, usually more candidate points are needed for the global proposal because candidates are not refined after each split. The local proposal refines the candidates after splits, 
and can potentially be more appropriate for deeper trees. A comparison of different algorithms on a Higgs boson dataset is given by Fig 3. 
We find that the local proposal indeed requires fewer candidates. 
The global proposal can be as accurate as the local one given enough candidates.

<img src='../../Other/img/xgboost2.png' style="width:650px;height:490px">

***

**Algorithm 2**: Approximate Algorithm for Split Finding

**for** $k=1$ **to** $m$ **do**

&emsp;&emsp;Propose $ S_k = \{s_{k1},s_{k2},\cdots,s_{kl} \} $ by percentiles on feature k 

&emsp;&emsp;Proposal can be done per tree(global), or per split(local)

**end**

**for** $k=1$ **to** $m$ **do**

&emsp;&emsp;$ G_{kv} \leftarrow = \sum_{j \in \{  j| s_{k,v} \geq \mathbf{x}_{jk} > s_{k, v-1} \}} g_j $

&emsp;&emsp;$ H_{kv} \leftarrow = \sum_{j \in \{  j| s_{k,v} \geq \mathbf{x}_{jk} > s_{k, v-1} \}} h_j $

**end**

Follow same step as in previous section to find max score only among proposed splits.

***


### Weighted Quantile Sketch           
&emsp;&emsp;One important step in the approximate algorithm is to propose candidate split points. 
Usually percentiles of a feature are used to make candidates distribute evenly on the data. Formally, 
let multi-set $ D_{k} = \{ (x_{1k}, h_1),(x_{2k}, h_2),\cdots,(x_{nk}, h_n) \} $represent the k-th feature values and second order gradient
statistics of each training instances. We can define a rank functions $r_{k}: \mathbb{R} \rightarrow[0,+\infty)$ as        
$$r_{k}(z)=\frac{1}{\sum_{(x, h) \in \mathcal{D}_{k}} h} \sum_{(x, h) \in \mathcal{D}_{k}, x<z} h \tag{8}$$       
which represents the proportion of instances whose feature value $k$ is smaller than $z$. The goal is to find candidate split points $ \{s_{k1},s_{k2},\cdots,s_{kl} \}  $such that         
$$\left|r_{k}\left(s_{k, j}\right)-r_{k}\left(s_{k, j+1}\right)\right|<\epsilon, \quad s_{k 1}=\min _{i} \mathbf{x}_{i k}, s_{k l}=\max _{i} \mathbf{x}_{i k} \tag{9} $$           
Here $ \epsilon$ is an approximation factor. Intuitively, this means that there is roughly $ 1/ \epsilon $ candidate points. Here each data point is weighted by $h_i$. 
To see why $h_i$ represents the weight, we can rewrite Eq (3) as             
$$\sum_{i=1}^{n} \frac{1}{2} h_{i}\left(f_{t}\left(\mathbf{x}_{i}\right) + g_{i} / h_{i}\right)^{2}+\Omega\left(f_{t}\right)+\text { constant } $$          
which is exactly weighted squared loss with labels $ - g_i / h_i$ and weights $h_i$. For large datasets, it is non-trivial to find candidate splits that satisfy the criteria.              


### Sparsity-aware Split Finding      
&emsp;&emsp;In many real-world problems, it is quite common for the input $x$ to be sparse. 
There are multiple possible causes for sparsity: 1)presence of missing values in the data;
2)frequent zero entries in the statistics;3)artifacts of feature engineering such as one-hot encoding; 
It is important to make the algorithm aware of the sparsity pattern in the data. In order to do so, 
we propose to add a default direction in each tree node, which is shown in Fig 4. 
When a value is missing in the sparse matrix $\mathbf{x}$, the instance is classified into the default direction.            

<img src='../../Other/img/xgboost3.png?raw=true' style="width:500px;height:250px">

&emsp;&emsp;There are two choices of default direction in each branch. 
The optimal default directions are learnt from the data. The algorithm is shown in Alg 3. 
The key improvement is to only visit the non-missing entries $I_k$. The presented algorithm treats the non-presence as a missing value and learns the best direction to handle missing values.        

***

**Algorithm 3**: Sparsity-aware Split Finding

**Input**: $I$,instance set of current node 

**input**:$I_{k}=\left\{i \in I | x_{i k} \neq \text { missing }\right\}$

**Input**: $d$,feature dimension

Also applies to the approximate setting, only collect statistics of non-missing entries into buckets

gain $\leftarrow 0 $

$G \leftarrow \sum_{i \in I} g_{i}, H \leftarrow \sum_{i \in I} h_{i}$

**for** $k = 1$ **to** $m$ **do**

&emsp;&emsp;//enumerate missing value goto right

&emsp;&emsp;$G_{L} \leftarrow 0, H_{L} \leftarrow 0$

&emsp;&emsp;**for** $j$ $in$ $sorted(I_k, \text{ascent order by} \, \mathbf{x}_{jk})$ **do**

&emsp;&emsp;&emsp;&emsp;$G_{L} \leftarrow G_{L}+g_{j}, \quad H_{L} \leftarrow H_{L}+h_{j}$

&emsp;&emsp;&emsp;&emsp;$G_{R} \leftarrow G-G_{L}, H_{R} \leftarrow H-H_{L}$

&emsp;&emsp;&emsp;&emsp;$\text {score} \leftarrow \max \left(\text {score}, \frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{G^{2}}{H+\lambda}\right)$

&emsp;&emsp;**end**

&emsp;&emsp;//enumerate missing value goto left

&emsp;&emsp;$G_{L} \leftarrow 0, H_{L} \leftarrow 0$

&emsp;&emsp;**for** $j$ $in$ $sorted(I_k, \text{ascent order by} \, \mathbf{x}_{jk})$ **do**

&emsp;&emsp;&emsp;&emsp;$G_{R} \leftarrow G_{R}+g_{j}, \quad H_{R} \leftarrow H_{R}+h_{j}$

&emsp;&emsp;&emsp;&emsp;$G_{L} \leftarrow G-G_{L}, H_{L} \leftarrow H-H_{R}$

&emsp;&emsp;&emsp;&emsp;$\text {score} \leftarrow \max \left(\text {score}, \frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{G^{2}}{H+\lambda}\right)$

&emsp;&emsp;**end**

**output**: Split and default direction with max gain

***